[langchain官方链接](https://python.langchain.com/docs/get_started/introduction)

**优秀的提示词**
- 【立角色】引导AI进入场景，赋予其身份

- 【问题】告诉AI你的困惑和问题，以及背景信息

- 【定目标】告诉AI你的需求，以及期望的目标

- 【补要求】告诉AI其他需要注意的事项，或者想要返回的内容

**设置环境变量key**

In [6]:
import os

os.environ["OPENAI_API_KEY"] = "sk-xxx"
os.environ["OPENAI_API_BASE"] = "https://api.chatanywhere.tech/v1"

### 提示词模版

#### PromptTemplate

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "请告诉我一个有关于{content}的{adjective}的笑话."
)
prompt_template.format(adjective="有趣的", content="小鸡")

#### ChatPromptTemplate

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一名Ai应用工程师. Your name is {name}."),
        ("human", "你好，你怎么样了?"),
        ("ai", "我很好，谢谢!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="Barry", user_input="请问你能干什么?")
print(messages)

<code>
[
  SystemMessage(content='你是一名Ai应用工程师. Your name is Barry.'), 
  HumanMessage(content='你好，你怎么样了?'), 
  AIMessage(content='我很好，谢谢!'), 
  HumanMessage(content='请问你能干什么?')
  ]
>
</code>

#### Message Prompts

In [ ]:
from langchain_core.prompts import ChatMessagePromptTemplate

prompt = "愿{subject}与你同在"

chat_message_prompt = ChatMessagePromptTemplate.from_template(
    role="Barry", template=prompt
)
chat_message_prompt.format(subject="星辰")

### 格式化提示词模版

#### fstring格式化模板

In [ ]:
##f-string是python内置的一种模板引擎，类似于js的模板字符串`${}`
from langchain.prompts import PromptTemplate

fstring_template = """给我讲一个{name}的{what}故事
哈哈哈
"""

prompt = PromptTemplate.from_template(fstring_template)

prompt.format(name="小六", what="开心")

#### jinja2格式化模板

In [ ]:
! pip install jinja2

In [ ]:
##Jinja2是一个灵活、高效的Python模板引擎，可以方便地生成各种标记格式的文档。
from langchain.prompts import PromptTemplate

jinja2_template = "给我讲一个{{name}}的{{what}}故事\n哈哈哈"
prompt = PromptTemplate.from_template(jinja2_template, template_format="jinja2")

prompt.format(name="小六", what="高兴")

### 组合提示词模版 Composition

#### String prompt composition
字符串提示词组合

In [ ]:
from langchain_core.prompts import PromptTemplate

# base_template = "请告诉我一个有关于{topic}的笑话。"
# template2 = "需要使用{language}语言"

# prompt = (
#   PromptTemplate.from_template(base_template) + template2
# )
# print(prompt)

# prompt.format(topic="运动", language="西班牙语")


prompt = (
    PromptTemplate.from_template("Tell me a joke about {topic}")
    + ", make it funny"
    + "\n\nand in {language}"
)

prompt.format(topic="sports", language="spanish")

**在LLM中使用**

In [7]:
import os
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
# from langchain.chat_models import ChatOpenAI

model = ChatOpenAI(
  openai_api_key=os.getenv("OPENAI_KEY"), 
  # temperature=0,
  # openai_api_base=os.getenv("OPENAI_API_BASE"),
  # openai_api_key=os.getenv("OPENAI_KEY")
)

chain = LLMChain(llm=model, prompt=prompt)
chain.run(topic="sports", language="spanish")

ValidationError: 2 validation errors for LLMChain
prompt
  Can't instantiate abstract class BasePromptTemplate without an implementation for abstract methods 'format', 'format_prompt' (type=type_error)
llm
  Can't instantiate abstract class BaseLanguageModel without an implementation for abstract methods 'agenerate_prompt', 'apredict', 'apredict_messages', 'generate_prompt', 'invoke', 'predict', 'predict_messages' (type=type_error)

#### Chat prompt composition
聊天提示词组合

In [ ]:
! pip install langchain_core
! pip install langchain_openai

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

prompt = SystemMessage(content="You are a nice pirate")
new_prompt = (
    prompt + HumanMessage(content="hi") + AIMessage(content="what?") + "{input}"
)
new_prompt.format_messages(input="i said hi")

model = ChatOpenAI()
chain = LLMChain(llm=model, prompt=new_prompt)
chain.run("i said hi")

#### PipelinePrompt

PipelinePrompt 由两个主要部分组成
- Final prompt: 返回的最终提示
- Pipeline prompts: 由字符串名称和提示模板组成的元组列表。将对每个提示模板进行格式化，然后将其作为具有相同名称的变量传递给未来的提示模板

In [ ]:
from langchain_core.prompts.pipeline import PipelinePromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.prompts import load_prompt

full_template = """{introduction}

{example}

{start}"""
full_prompt = PromptTemplate.from_template(full_template)

introduction_template = """You are impersonating {person}."""
introduction_prompt = PromptTemplate.from_template(introduction_template)

example_template = """Here's an example of an interaction:

Q: {example_q}
A: {example_a}"""
example_prompt = PromptTemplate.from_template(example_template)

start_template = """Now, do this for real!

Q: {input}
A:"""
start_prompt = PromptTemplate.from_template(start_template)

input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start", start_prompt),
]
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, pipeline_prompts=input_prompts
)

print(pipeline_prompt.input_variables)

print(
    pipeline_prompt.format(
        person="Elon Musk",
        example_q="What's your favorite car?",
        example_a="Tesla",
        input="What's your favorite social media site?",
    )
)

#### 使用本地文件管理 PromptTemplate

In [4]:
from langchain_core.prompts import load_prompt

prompt = load_prompt('simple_prompt.json')
print(prompt.format(name="小蓝",what="励志的"))

# 最新版本不支持解析方式
# prompt = load_prompt("prompt_with_output_parser.json")
# prompt.output_parser.parse(
#     "George Washington was born in 1732 and died in 1799.\nScore: 1/2"
# )


给我讲一个关于小蓝的励志的故事
